In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import os
import sys


###########################################################################################
###########################################################################################
#Definition of functions
###########################################################################################
###########################################################################################


###########################################################################################
#calculate distance between original and reflected torso
###########################################################################################

def calculate_asymmetry_dist(source_temp, target_temp, sign_dist_metric):
    #o3d.visualization.draw_geometries([source_temp, target_temp])

    #calculate distance between the 2 torso's
    dists = source_temp.compute_point_cloud_distance(target_temp)
    dists = np.asarray(dists)
    dists_norm = copy.deepcopy(dists) 

    #Paint point color according to distance---------------------
    lendists = len(dists)
    maxdist = max(dists)
    np_colors = np.random.rand(lendists, 3)  #create 3d float array for colors

    #assign colors normalized to 1 to RGB color array
    for i in range(0, lendists):
        #if sign_dist_metric[i] > 0:
            dists_norm[i] = dists[i]/maxdist
            np_colors[i,0] = dists_norm[i]
            np_colors[i,1] = 1-dists_norm[i]
            np_colors[i,2] = 1-dists_norm[i]
        #elif sign_dist_metric[i] < 0:
        #    np_colors[i,0] = 0
        #    np_colors[i,1] = 0
        #    np_colors[i,2] = 0

    #create temporary PointCloud pcd for visualization and assign points as well as colors 
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(source_temp.points)
    pcd.colors = o3d.utility.Vector3dVector(np_colors)
    o3d.visualization.draw_geometries([pcd])

    #------------------------------------------------------------
    #Print Original and PointCloud with removed points (>9mm)
    #------------------------------------------------------------

    ind = np.where(sign_dist_metric > 9)[0]
    pcd_threshold = source_temp.select_by_index(ind)
    pcd_threshold.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([pcd_threshold, source_temp])

    print("Print the distances of the first 10 points")
    print(np.asarray(dists)[:10])

    #---------------------------------------------
    #Save all asymmetry distances in text file
    #---------------------------------------------
    print('Save distances in float array')
    #---------------------------------------------
    lendists = len(dists)
    f = open("distances.dat", "w+")
    for i in range(0, lendists):
        f.write(str(sign_dist_metric[i]))
        f.write("\n")  
    #----------------------------------------------

#---------------------------------------------------------------------------


###########################################################################################
#calculate statistics
###########################################################################################

def calculate_statistics(pcd_01, sign_dist_metric):

    pcd_01.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=30, max_nn=30))

    o3d.visualization.draw_geometries([pcd_01],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)

    print("any normal vector")
    print(print(pcd_01.normals[10]))

    pcd_points_01 = np.asarray(pcd_01.points)
    #vec_normal = pcd_points_01[1,:]
    nr = len(pcd_points_01)

    
    distmax = -100000

    #Bestimmt die Oberflaechen Rotation am Punkt mit der groessten Asymmetrie
    #---------------------
    for i in range(nr):

        if sign_dist_metric[i] > distmax: 
            distmax = sign_dist_metric[i]
            vec_normal = pcd_01.normals[i]

    #vec_normal = np.asarray(vec_normal)
    print("normal vectors")
    print(vec_normal)

    #Calculate surface rotation relative to sagittal plane
    ortho_vec = copy.deepcopy(vec_normal)
    ortho_vec[0] = 0
    ortho_vec[1] = 1
    ortho_vec[2] = 0

    print("ortho_vec")
    print(ortho_vec)

    #calculate vector norm
    vec_normal_norm  = math.sqrt( vec_normal[0]**2 + vec_normal[1]**2 + vec_normal[2]**2 )
    ortho_vec_norm = math.sqrt( ortho_vec[0]**2 + ortho_vec[1]**2 + ortho_vec[2]**2 )

    print(vec_normal_norm)
    print(ortho_vec_norm)

    #dot product of vectors
    dot_product_vectors = vec_normal[0]*ortho_vec[0] + vec_normal[1]*ortho_vec[1] + vec_normal[2]*ortho_vec[2] 
    print(dot_product_vectors)

    #Temp = 3.14 * ( (dot_product_vectors) / (vec_normal_norm * ortho_vec_norm) / 180 )
    Temp = ( (dot_product_vectors) / (vec_normal_norm * ortho_vec_norm) )
    surf_angle = math.acos( Temp )


    print("This is the surface rotation at the max asymmetry location !")
    print(surf_angle)


###########################################################################################
#ICP matching for original and reflection
###########################################################################################

###########################################################################################

#type(source)

#------------------------------------------------------------------------
def draw_registration_result_init(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


#------------------------------------------------------------------------
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

    calculate_asymmetry_dist(source_temp, target_temp, sign_dist_metric)


#--------------------------------------------------------------------------
def point_to_point_icp(source, target, threshold, trans_init):
    print("Apply point-to-point ICP")
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())

    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation, "\n")
    draw_registration_result_init(source, target, reg_p2p.transformation)

#-------------------------------------------------------------------------
def point_to_plane_icp(source, target, threshold, trans_init):
    print("Apply point-to-plane ICP")
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())

    print(reg_p2l)
    print("Transformation is:")
    print(reg_p2l.transformation, "\n")
    draw_registration_result(source, target, reg_p2l.transformation)

#---------------------------------------------------------------------------
###########################################################################################


###########################################################################################
###########################################################################################
#End of function definition
###########################################################################################
###########################################################################################


###########################################################################################

#import point cloud

#pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/model_clean.ply")
pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/test_torso_02.ply")
pcd_01_reflect = copy.deepcopy(pcd_01)

#pcd_02.paint_uniform_color([2, 0.706, 0])
#o3d.visualization.draw_geometries([pcd_01, pcd_02])

#mesh = o3d.io.read_triangle_mesh("C:/Users/User/Open3D_Folder/Data/test_torso_02.ply")
#print("Try to render a mesh with normals (exist: " +
#          str(mesh.has_vertex_normals()) + ") and colors (exist: " +
#          str(mesh.has_vertex_colors()) + ")")
#o3d.visualization.draw_geometries([mesh])
#mesh.compute_vertex_normals()
#print(np.asarray(mesh.triangle_normals))
#o3d.visualization.draw_geometries([mesh])

#vertex_normal = mesh.triangle_normals[100]
#print("Print any vertex normal")
#print(vertex_normal)

#nr = len(mesh.triangle_normals)
#print(nr)

#nr1 = len(mesh.triangles)
#print(nr1)


###########################################################################################


#Estimate vertex normals

#it is important to chose the right radius (not to small >5) to gain realistic surface normals
pcd_01.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=10, max_nn=30))

#o3d.visualization.draw_geometries([pcd_01],
#                                  zoom=0.3412,
#                                  front=[0.4257, -0.2125, -0.8795],
#                                  lookat=[2.6172, 2.0475, 1.532],
#                                  up=[-0.0694, -0.9768, 0.2024],
#                                  point_show_normal=True)

print("Print a normal vector of the 0th point")
print(pcd_01.normals[0])

print("Print the normal vectors of the first 10 points")
print(np.asarray(pcd_01.normals)[:10, :])

nr1 = len(pcd_01.normals)
print(nr1)

##############################################################################################
##############################################################################################
##############################################################################################

###########################################################################################
#oriented bounding box

#obj1 = pcd_01.get_axis_aligned_bounding_box()
#obj1.color = (1, 0, 0)
#obj2 = pcd_01.get_oriented_bounding_box()
#obj2.color = (0, 0, 1)
#o3d.visualization.draw_geometries([pcd_01, obj1, obj2])

##########################################################################################
#calculate centre of point cloud
##########################################################################################

pcd_points_01 = np.asarray(pcd_01.points)
pcd_centre_01 = pcd_points_01[1,:] + pcd_points_01[2,:]
pcd_centre_01[0] = 0
pcd_centre_01[1] = 0
pcd_centre_01[2] = 0

#calculate centre of torso point cloud####################################################

nr = len(pcd_points_01)
print(nr)

#calculate centre
#---------------------
for i in range(nr):
  pcd_centre_01 = pcd_centre_01 + pcd_points_01[i,:]
#---------------------
pcd_centre_01 = pcd_centre_01/nr

#centre point cloud # origin (0,0,0) is placed in torso centre
#---------------------
for i in range(nr):
  pcd_points_01[i,:] = pcd_points_01[i,:] - pcd_centre_01

print(np.asarray(pcd_points_01)[0,:])

##########################################################################################
#make copy of point cloud and perform reflection about sagittal plane
##########################################################################################

#make copy of torso object
#pcd_points_01_reflect = pcd_points_01
pcd_points_01_reflect = pcd_points_01.copy()
#print(np.asarray(pcd_centre_01)[:])
    #source_temp = copy.deepcopy(source)
    #target_temp = copy.deepcopy(target)

#do reflection about sagittal plane
for i in range(nr):
  temp = pcd_points_01[i,0]
  pcd_points_01_reflect[i,0] = -temp

print(np.asarray(pcd_points_01)[0,:])
print(np.asarray(pcd_points_01_reflect)[0,:])


##########################################################################

##########################################################################
##########################################################################
#Assign centred and reflected points again to point cloud objects
##########################################################################

pcd_01.points = o3d.utility.Vector3dVector(pcd_points_01)
pcd_01_reflect.points = o3d.utility.Vector3dVector(pcd_points_01_reflect)

##########################################################################
##########################################################################

##########################################################################

###########################################################################################
#------------------------------------------------------------------------------------------
#Detect which points of the reflection are positioned inside the original volume
#points with shorter distance to the centre are inside
#------------------------------------------------------------------------------------------
###########################################################################################

#Variablennamen ueberpruefen

sign_dist = pcd_points_01[:,0].copy()
sign_dist_metric = pcd_points_01[:,0].copy()

for i in range(nr):
    distmin = 100000

    #Distanzfunktion von Open3D nutzen anstatt Schleife
    for k in range(math.floor(nr/30)): #werte jeden 30. Punkt aus
        
        j = k*30
#        #calculate manually the distance between original and reflection
        dists_man = math.sqrt( (pcd_points_01[i,0] - pcd_points_01_reflect[j,0])**2 + (pcd_points_01[i,1] - pcd_points_01_reflect[j,1])**2 + (pcd_points_01[i,2] - pcd_points_01_reflect[j,2])**2 )

        if dists_man < distmin: 
            distmin = dists_man
            near_point = pcd_points_01_reflect[j,:]
            sign_dist_metric[i] = dists_man

    #Calculate the distance between torso points and centre of mass for original and reflection
    dist_orig = math.sqrt( (pcd_points_01[i,0] - pcd_centre_01[0])**2 + (pcd_points_01[i,1] - pcd_centre_01[1])**2 + (pcd_points_01[i,2] - pcd_centre_01[2])**2 )
    dist_refl = math.sqrt( (near_point[0] - pcd_centre_01[0])**2 + (near_point[1] - pcd_centre_01[1])**2 + (near_point[2] - pcd_centre_01[2])**2 )

    if dist_orig > dist_refl:
        sign_dist[i] = 1
    else: 
        sign_dist[i] = 0                            #Vorzeichen der Asymmetrie Distanz +/- zwischen Original und Reflection
        sign_dist_metric[i] = - sign_dist_metric[i] #Metrischer Abstand [mm] zwischen Original und Reflection

print("Print the signed distances of the first 10 points")
print(np.asarray(sign_dist)[:])


###########################################################################################
#Start of matching
###########################################################################################

#if __name__ == "__main__":
    #pcd_data = o3d.data.DemoICPPointClouds()
    #source = o3d.io.read_point_cloud(pcd_data.paths[0])
    #source_final = copy.deepcopy(source)
    #target = o3d.io.read_point_cloud(pcd_data.paths[1])
    # Input point-clouds
    #pcd_points_01_reflect = pcd_points_01
#Wichtig ist, dass hier 3D point cloud objekte fuer das Matching uebergeben werden
#nur die Punktkoordinaten reichen nicht aus
source = copy.deepcopy(pcd_01)
target = copy.deepcopy(pcd_01_reflect)
threshold = 0.105
    #trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
    #                         [-0.139, 0.967, -0.215, 0.7],
    #                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
    
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
    


    #draw_registration_result_init(source, target, trans_init)
    #print("Initial alignment")

evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)

print(evaluation, "\n")
    #point_to_point_icp(source, target, threshold, trans_init)
point_to_plane_icp(source, target, threshold, trans_init)

################################################################################
#Start of statitistic calculation based on asymmetry map
################################################################################

calculate_statistics(pcd_01, sign_dist_metric)
    

    








Print a normal vector of the 0th point
[-0.68702759  0.26344667 -0.67719196]
Print the normal vectors of the first 10 points
[[-0.68702759  0.26344667 -0.67719196]
 [ 0.9423888  -0.20940467  0.26086977]
 [-0.30663043  0.30873754 -0.90036599]
 [-0.35018376  0.28279174 -0.89297266]
 [-0.37122326  0.3477161  -0.86098014]
 [-0.299552    0.27272155 -0.91427105]
 [-0.29105095  0.25845988 -0.92113399]
 [-0.24142926  0.289388   -0.92626481]
 [-0.19812153  0.27697657 -0.94022968]
 [-0.23444493  0.17828925 -0.95564037]]
36969
36969
[ 142.35752066 -170.78948431  -13.91816196]
[ 142.35752066 -170.78948431  -13.91816196]
[-142.35752066 -170.78948431  -13.91816196]
Print the signed distances of the first 10 points
[0. 0. 1. ... 0. 0. 0.]
RegistrationResult with fitness=5.139441e-04, inlier_rmse=7.348656e-02, and correspondence_set size of 19
Access transformation to get result. 

Apply point-to-plane ICP
RegistrationResult with fitness=7.032919e-04, inlier_rmse=6.630068e-02, and correspondence_set s